# Global Configuration

In [1]:
# THIS CELL IMPORTS ALL THE NECESSARY LIBRARIES TO RUN THE CODE

# Is_To_Run_On_Cloud = True
from keras import backend as K
import keras
import json
# Is_HPC = True
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
# if(Is_HPC):
#     pass
# else:
import pandas as pd
import tensorflow as tf

import numpy as np
from keras.layers.core import Flatten, Dense,Dropout
from keras import Model
from keras.preprocessing.image import  ImageDataGenerator
from keras.utils import multi_gpu_model
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ModelCheckpoint
from keras.models import model_from_json
import time
from time import sleep
import gc
from datetime import datetime
from enum import Enum
import os
import cv2
from tensorflow.python.client import device_lib
# if(Is_To_Run_On_Cloud):
#     from keras import keras_applications as K_Models
# else:
from keras import applications as K_Models
# Is_Running_On_Local_Desktop = True
# FAA_HPC = False
from classification_models.resnet import ResNet18, ResNet34, preprocess_input

    
# if(Training_Gauges["True Airspeed(kn)"])

Using TensorFlow backend.


In [18]:
# THIS CELL CREATES LENNET MODEL ARCHITECTURE

PADDING = "SAME" # "VALID"
def LenNet(input_size, output_classes):
    # Vanilla CNN. Play around with architecture.
    model = Sequential()
    model.add(Conv2D(filters = 64, 
                     kernel_size = 9, 
                     strides = 1, 
                     activation = 'relu', 
                     padding=PADDING,
                     input_shape = (input_size[0],input_size[1],input_size[2])))
    model.add(Conv2D(filters = 64, 
                     kernel_size = 7, 
                     strides = 1, 
                     activation = 'relu',
                     padding=PADDING,
                    ))
    model.add(Conv2D(filters = 64, 
                     kernel_size = 5, 
                     strides = 1, 
                     activation = 'relu', 
                     padding=PADDING,
                     ))
    model.add(MaxPooling2D(pool_size = 5, strides = 1))
    model.add(Conv2D(filters = 64, 
                 kernel_size = 3,
                 strides = 1,
                 activation = 'relu',
                 padding=PADDING,
                ))
    model.add(Conv2D(filters = 64, 
                 kernel_size = 3,
                 strides = 1,
                 padding=PADDING,
                 activation = 'relu'
                 ))
    model.add(Conv2D(filters = 64, 
                 kernel_size = 3,
                 strides = 1,
                 padding=PADDING,
                 activation = 'relu'
                 ))
    model.add(MaxPooling2D(pool_size = 5, strides = 1))
    model.add(Conv2D(filters = 64, 
                     kernel_size = 3,
                     strides = 1,
                     activation = 'relu',
                     padding=PADDING
                     ))
    model.add(Conv2D(filters = 64, 
                     kernel_size = 3,
                     strides = 1,
                     activation = 'relu',
                     padding=PADDING
                    ))
    model.add(MaxPooling2D(pool_size = 3, strides = 2))
    model.add(Dropout(0.25))
    model.add(Flatten())
#     model.add(Dense(64, activation='relu'))
#     model.add(Dropout(0.5))
    model.add(Dense(output_classes, activation='softmax', name='softmax'))
    return model

In [3]:
##### DEFINE THE FUNCTIONS USED IN THE FINAL FUNCTION TO DO TRAINING ######

##############################################################################################
# 1. Load the correct model.
# INPUT -> SELECTED_MODEL (must be from list of available models)
# OUTPUT -> ML_Model
def Load_Model():
    global ML_Model
    if(SELECTED_MODEL == Available_Models.VGG16):
        ML_Model = K_Models.VGG16(include_top=INCLUDE_TOP, weights=INITIALIZATION_WIEGHTS, 
                                                              input_tensor=INPUT_TENSOR, input_shape=INPUT_SIZE,
                                                              pooling=POOLING, classes=OUTPUT_CLASSES)
    elif(SELECTED_MODEL == Available_Models.VGG19):
        ML_Model = K_Models.VGG19(include_top=INCLUDE_TOP, weights=INITIALIZATION_WIEGHTS, 
                                                              input_tensor=INPUT_TENSOR, input_shape=INPUT_SIZE,
                                                              pooling=POOLING, classes=OUTPUT_CLASSES)
    elif(SELECTED_MODEL == Available_Models.InceptionV3):
        ML_Model = K_Models.InceptionV3(include_top=INCLUDE_TOP, weights=INITIALIZATION_WIEGHTS, 
                                                              input_tensor=INPUT_TENSOR, input_shape=INPUT_SIZE,
                                                              pooling=POOLING, classes=OUTPUT_CLASSES)
    elif(SELECTED_MODEL == Available_Models.ResNet50):
        ML_Model = K_Models.ResNet50(include_top=INCLUDE_TOP, weights=INITIALIZATION_WIEGHTS, 
                                                              input_tensor=INPUT_TENSOR, input_shape=INPUT_SIZE,
                                                              pooling=POOLING, classes=OUTPUT_CLASSES)
    elif(SELECTED_MODEL == Available_Models.ResNet18):
        ML_Model = ResNet18(input_shape=INPUT_SIZE, weights=INITIALIZATION_WIEGHTS, include_top=INCLUDE_TOP)

    elif(SELECTED_MODEL == Available_Models.ResNet34):
        ML_Model = ResNet34(input_shape=INPUT_SIZE, weights=INITIALIZATION_WIEGHTS, include_top=INCLUDE_TOP)
        
    elif(SELECTED_MODEL == Available_Models.DenseNet121):
        ML_Model = K_Models.DenseNet121(include_top=INCLUDE_TOP, weights=INITIALIZATION_WIEGHTS, 
                                                              input_tensor=INPUT_TENSOR, input_shape=INPUT_SIZE,
                                                              pooling=POOLING, classes=OUTPUT_CLASSES)
    elif(SELECTED_MODEL == Available_Models.DenseNet169):
        ML_Model = K_Models.DenseNet169(include_top=INCLUDE_TOP, weights=INITIALIZATION_WIEGHTS, 
                                                              input_tensor=INPUT_TENSOR, input_shape=INPUT_SIZE,
                                                              pooling=POOLING, classes=OUTPUT_CLASSES)
    elif(SELECTED_MODEL == Available_Models.DenseNet201):
        ML_Model = K_Models.DenseNet201(include_top=INCLUDE_TOP, weights=INITIALIZATION_WIEGHTS, 
                                                              input_tensor=INPUT_TENSOR, input_shape=INPUT_SIZE,
                                                              pooling=POOLING, classes=OUTPUT_CLASSES)
    elif(SELECTED_MODEL == Available_Models.MobileNet):
        ML_Model = K_Models.MobileNet(include_top=INCLUDE_TOP, weights=INITIALIZATION_WIEGHTS, 
                                                              input_tensor=INPUT_TENSOR, input_shape=INPUT_SIZE,
                                                              pooling=POOLING, classes=OUTPUT_CLASSES)

    elif(SELECTED_MODEL == Available_Models.InceptionResNetV2):
        ML_Model = K_Models.InceptionResNetV2(include_top=INCLUDE_TOP, weights=INITIALIZATION_WIEGHTS, 
                                                              input_tensor=INPUT_TENSOR, input_shape=INPUT_SIZE,
                                                              pooling=POOLING, classes=OUTPUT_CLASSES)
    elif(SELECTED_MODEL == Available_Models.NASNetLarge):
        ML_Model = K_Models.NASNetLarge(include_top=INCLUDE_TOP, weights=INITIALIZATION_WIEGHTS, 
                                                              input_tensor=INPUT_TENSOR, input_shape=INPUT_SIZE,
                                                            pooling=POOLING, classes=OUTPUT_CLASSES)
    elif(SELECTED_MODEL == Available_Models.Xception):
        ML_Model = K_Models.Xception(include_top=INCLUDE_TOP, weights=INITIALIZATION_WIEGHTS, 
                                                              input_tensor=INPUT_TENSOR, input_shape=INPUT_SIZE,
                                                              pooling=POOLING, classes=OUTPUT_CLASSES)
    elif(SELECTED_MODEL == Available_Models.NASNetMobile):
        ML_Model = K_Models.NASNetMobile(include_top=INCLUDE_TOP, weights=INITIALIZATION_WIEGHTS, 
                                                              input_tensor=INPUT_TENSOR, input_shape=INPUT_SIZE,
                                                              pooling=POOLING, classes=OUTPUT_CLASSES)
    elif(SELECTED_MODEL == Available_Models.Lenet_Style):
        ML_Model = LenNet(input_size= INPUT_SIZE, output_classes=OUTPUT_CLASSES)
        return
    else:
        print("NO MODEL IDENTIFIER MATCHED")
        
    # Add the output layer after the model architecture
    if(POOLING is None):
        last_layer = Flatten()(ML_Model.output)
    else:
        last_layer = ML_Model.output
        if(SELECTED_MODEL ==  Available_Models.ResNet18 or SELECTED_MODEL == Available_Models.ResNet34):
            last_layer = keras.layers.GlobalAveragePooling2D()(ML_Model.output)
        
    if(len(Dense_Layers_Neurons) > 0):
        for n in Dense_Layers_Neurons:
            last_layer = Dense(n, activation='relu')(last_layer)
            if(Is_To_Add_Dropout):
                last_layer =  Dropout(Dropout_Rate)
    else:
        if(Is_To_Add_Dropout):
    #         for neurons in Dense_Layers:
    #             last_layer = Dense(neurons, activation='relu')(last_layer)
            last_layer =  Dropout(Dropout_Rate)(last_layer)
        
    last_layer = Dense(OUTPUT_CLASSES, activation=FINAL_LAYER_CLASSIFICATION_FUCNTION,
                       name='softmax')(last_layer)
    
    # This is the output of this function, feeds into function 2
    ML_Model = Model(ML_Model.input, last_layer)
    
##############################################################################################

##############################################################################################
# 2. Regularize Layers.
# INPUT -> ML_Model, REGULARIZE_EARLY_LAYERS_WIHT_PERCENT, REGULARIZE_EARLY_LAYERS_WITH_RATIO
# OUTPUT -> ML_Model with regularized layers
def Add_Conv_Regularizer():
    global ML_Model
#     alpha = 0.0001  # weight decay coefficient
    if(REGULARIZE_EARLY_LAYERS_WIHT_PERCENT <= 0):
        # Do Not Regularize Any Layers
        print("Zero layers regularized")
    else:
        # Regularizing Models Layers
        regularized_layers_count = int(len(ML_Model.layers) * REGULARIZE_EARLY_LAYERS_WIHT_PERCENT)
        print("Regularize {0} early layers".format(regularized_layers_count))
        for layer in ML_Model.layers[:regularized_layers_count]:
            
            if isinstance(layer, keras.layers.Conv2D) or isinstance(layer, keras.layers.Dense):
#                 print("Before Regularization Added=" + str(layer.losses))
                layer.add_loss(keras.regularizers.l2(REGULARIZE_EARLY_LAYERS_WITH_RATIO)(layer.kernel))
#                 print("Layer Name=", layer.name)
#                 print("After Regularization Added=" + str(layer.losses))
            if hasattr(layer, 'bias_regularizer') and layer.use_bias:
                layer.add_loss(keras.regularizers.l2(REGULARIZE_EARLY_LAYERS_WITH_RATIO)(layer.bias))
            
##############################################################################################

##############################################################################################
# 3. Determines if you want to freeze layers or not.
# INPUT -> ML_Model, FREEZE_EARLY_LAYERS_WIHT_PERCENT
# OUTPUT -> ML_Model with frozen layers
def Freeze_Model_Parameters():#ML_Model, FREEZE_EARLY_LAYERS_WIHT_PERCENT):
    global ML_Model
    if(FREEZE_EARLY_LAYERS_WIHT_PERCENT <= 0):
        # Do Not Freeze any Layers
        print("Zero layers freezed")
    else:
        # Freezing Models Layers
        freezed_layers_count = int(len(ML_Model.layers) * FREEZE_EARLY_LAYERS_WIHT_PERCENT)
        print("Freeze {0} early layers".format(freezed_layers_count))
        for layer in ML_Model.layers[:freezed_layers_count]:
            layer.trainable = False    

##############################################################################################

##############################################################################################
# 4. Compile the Model.
# INPUT -> ML_Model, LEARNING_RATE
# OUTPUT -> P_ML_Model (compiled model)
def Compile_Model():
    global P_ML_Model
    global ML_Model

    if(AVAILABLE_GPUs > 1):
        print("Compiling multi GPU models")
        P_ML_Model = multi_gpu_model(ML_Model, AVAILABLE_GPUs)
    else:
        print("Compiling single GPU model")
        P_ML_Model = ML_Model
    
    # set the adam optimizer
    adam = Adam(lr=LEARNING_RATE, beta_1=0.9, beta_2=0.999,
            epsilon=None, decay=1e-6, amsgrad=False)
    
    ##### THIS IS THE OUTPUT #####
    P_ML_Model.compile(loss="categorical_crossentropy", optimizer=adam, metrics=['accuracy'])
    
##############################################################################################

##############################################################################################
# 5. This loads in the data from the correct folder. 
# INPUT -> Is_To_Apply_Data_Augmentation, INPUT_SIZE, BATCH_SIZE
# OUTPUT -> TRAIN_GENERATOR, VALID_GENERATOR, STEP_SIZE_TRAIN, STEP_SIZE_VALID. These get fed into function 7
def Create_Data_Loader():
    global TRAIN_GENERATOR
    global VALID_GENERATOR
    global STEP_SIZE_TRAIN
    global STEP_SIZE_VALID
    
    if(Is_To_Apply_Data_Augmentation):
        # Apply Data Augmentation to the input images
        train_datagen = ImageDataGenerator(
            rescale=1./255,
#             rotation_range=40,
#             width_shift_range=0.2,
#             height_shift_range=0.2,
#             shear_range=0.2,
#             zoom_range=[-0.2, 0.2],
#             horizontal_flip=True,
#             fill_mode='nearest'
        )
        # normalize the validation dataset
        validation_datagen = ImageDataGenerator(
            rescale=1./255)
    else:
        # normalize the input data for training and validation
        train_datagen = ImageDataGenerator(rescale=1./255)
        validation_datagen = ImageDataGenerator(rescale=1./255)

    # Create the TRAINING GENERATOR
    TRAIN_GENERATOR = train_datagen.flow_from_directory(TRAINING_DIRECTORY_PATH,  # this is the target directory
                classes =CLASS_MAPPINGS,
                target_size=(INPUT_SIZE[0], INPUT_SIZE[1]),  # all images will be resized 
                batch_size=BATCH_SIZE,
                color_mode=COLOR_MODE,
                shuffle=SHUFFLE,
                seed=SEED,
                class_mode='categorical') 
    print("TRAINING_DIRECTORY_PATH=",TRAINING_DIRECTORY_PATH)
    print("TRAIN_GENERATOR.class_indices =", TRAIN_GENERATOR.class_indices)
    
    # Create the VALIDATION GENERATOR
    VALID_GENERATOR = validation_datagen.flow_from_directory(
                VALIDATION_DIRECTORY_PATH,
                classes =CLASS_MAPPINGS,
                target_size=(INPUT_SIZE[0], INPUT_SIZE[1]),
                batch_size=BATCH_SIZE,
                color_mode=COLOR_MODE,
                seed=SEED,
                class_mode='categorical')
    print("VALIDATION_DIRECTORY_PATH=",VALIDATION_DIRECTORY_PATH)
    print("TRAIN_GENERATOR.class_indices =", VALID_GENERATOR.class_indices)

    # Calculate STEP SIZE for Training and Validation
    STEP_SIZE_TRAIN=TRAIN_GENERATOR.n//TRAIN_GENERATOR.batch_size
    STEP_SIZE_VALID=VALID_GENERATOR.n//VALID_GENERATOR.batch_size
    print("STEP_SIZE_TRAIN=",STEP_SIZE_TRAIN)
    print("STEP_SIZE_VALID=",STEP_SIZE_VALID)
##############################################################################################    

##############################################################################################
# 6. Create the paths to where the data will be saved. 
# INPUT -> SELECTED_MODEL, C_Guage, Store_Trained_ML_Model_Info_At
# OUTPUT -> The paths are set to where the info will be saved
def Get_Model_Storage_Info():
    global Model_Name
    global Weights_Path
    global History_Path
    global Model_Path

    Model_Name = "{0}_{1}".format(
        SELECTED_MODEL, C_Guage)
    
    Weights_Path = "{0}/CheckPoint_{1}.hdf5".format(
                    Store_Trained_ML_Model_Info_At, Model_Name)
    
    History_Path = "{0}/{1}_history".format(Store_Trained_ML_Model_Info_At, 
                                            Model_Name)

    Model_Path = "{0}/{1}".format(Store_Trained_ML_Model_Info_At,
                                                          Model_Name)
    
    # PRINT THE PATHS
    print("Model_Name=", Model_Name)
    print("Weights_Path=", Weights_Path)
    print("History_Path=", History_Path)
    print("Model_Path=", Model_Path)
##############################################################################################

##############################################################################################
# 7. Do the actual training of the model
# INPUT -> Outputs from FUNCTION 5, 
# OUTPUT -> History file
def Do_Training():

    global History
    # Schedule a Learning Rate
    #lr_scheduler = LearningRateScheduler(lr_schedule)
    # Keep track of the model at each epoch and only save the model if it improves from last epoch
    checkpoint = ModelCheckpoint(Weights_Path, monitor=Monitor, verbose=1, save_best_only=True, mode=Mode)
    # set the early stopping criteria, EARLY STOPPING PATIENCE
    early_stopping = EarlyStopping(monitor=Monitor, min_delta=0, patience=EARLY_STOPPING_PATIENCE, verbose=0, 
                                       mode=Mode)

    #     lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1), cooldown=0, patience=5, min_lr=0.5e-6)
    print("CLASSES_WEIGTHAGES=", CLASSES_WEIGTHAGES)
    callbacks_list = [early_stopping,  checkpoint]#, lr_scheduler]
    
    # DO THE TRAINING
    History = P_ML_Model.fit_generator(
            TRAIN_GENERATOR,
            steps_per_epoch=STEP_SIZE_TRAIN,
            epochs=EPOCHs,
            validation_data=VALID_GENERATOR,
            validation_steps=STEP_SIZE_VALID, 
             callbacks=callbacks_list,
            class_weight = CLASSES_WEIGTHAGES)
##############################################################################################

##############################################################################################
# 8. Saves the info as a .json file?
# INPUT -> History_Path
# OUTPUT -> Model is Saved
def Save_Model_Info():
    with open('{0}.json'.format(History_Path), 'w') as f:
        json.dump(History.history, f)

    pd.DataFrame(History.history).to_csv("{0}.csv".format(History_Path))
    try:
        model_yaml = ML_Model.to_yaml()
        with open("{0}.yaml".format(Model_Path), "w") as yaml_file:
            yaml_file.write(model_yaml)
    except Exception as eExcep:
        print(str(eExcep))
    model_json = ML_Model.to_json()
    with open("{0}.json".format(Model_Path), "w") as json_file:
        json_file.write(model_json)
#     ML_Model.save(Model_Path)

##############################################################################################

##############################################################################################
# 9. This is for evaluating the test set. I'll be doing this with my own function instead. 
def Evaluate_Model():
    
    global TEST_CSV
    try:
        TEST_CSV["{0}_Prd_{1}_Selected_Model_{2}".format(C_Guage,Model_Name, SELECTED_MODEL)] = None
        try:
            for index, row in TEST_CSV.iterrows():
#                 try:
                if(index != 0 and index % 2500 == 0):
                    print("Predicted Examples Count={0}".format(index))
#                     break
                for test_img_path in TEST_CSV["ML_File_Path_For_TrainValTest"][index].split(","):
#                     print("test_img_path=", test_img_path)
                    try:
                        test_img_path = test_img_path.replace("./", "{0}/".format(Dell_Path))
    #                     print("test_img_path=", test_img_path)
                        test_img = cv2.imread(test_img_path, cv2.IMREAD_COLOR)
                        test_img = test_img.astype("float32") / 255
                        #                     test_img = np.expand_dims(test_img, -1)
                        test_img = cv2.resize(test_img, (INPUT_SIZE[0], 
                                                           INPUT_SIZE[1]))
                        test_img = np.reshape(test_img,(1,test_img.shape[0], 
                                    test_img.shape[1], test_img.shape[2]))
                        prediction = np.argmax(ML_Model.predict(test_img, 
                                                            batch_size=1))
                        TEST_CSV["{0}_Prd_{1}_Selected_Model_{2}".format(C_Guage,Model_Name, SELECTED_MODEL)][index] = prediction
                        #                 if(TEST_CSV["{0}_Prd_{1}".format(C_Guage,Model_Name)] = None)
                        #                 TEST_CSV["{0}_Prd_{1}".format(C_Guage,Model_Name)][index] = prediction
    #                     print("Prediction=" + str(prediction))
                    except Exception as eERRoR:
                        TEST_CSV["{0}_Prd_{1}_Selected_Model_{2}".format(C_Guage,Model_Name, SELECTED_MODEL)][index] = -1
                    break
#                 except Exception as eError:
#                     TEST_CSV["{0}_Prd_{1}".format(C_Guage, Model_Name)][index] = "Error"
#                     print("Some Error Occurred While Prediction." + str(eError))
#                     continue
        except Exception as eError:
            TEST_CSV["{0}_Prd_{1}_Selected_Model_{2}".format(C_Guage,Model_Name, SELECTED_MODEL)][index] = "Error"
            print("Some Error Occurred While Prediction." + str(eError))

    except Exception as eError:
        print("Some Error Occurred While Evaluating the Model." + str(eError))
    
##############################################################################################



##############################################################################################
# This is the main model training function.
def Initiate_Model_Training():
    global ML_Model
    global P_ML_Model
    
    print("SELECTED MODEL #" + str(SELECTED_MODEL))
    if(AVAILABLE_GPUs > 1):
        with tf.device('/cpu:0'):
            Load_Model()
    else:
        Load_Model()
    Add_Conv_Regularizer()
    Freeze_Model_Parameters()
    Compile_Model()
    Create_Data_Loader()
    Get_Model_Storage_Info()
    ML_Model.summary()
    Do_Training()
    Save_Model_Info()
#   Evaluate_Model()

    #Release Resources
    ML_Model = None
    P_ML_Model = None
    gc.collect()
    K.clear_session()
    sleep(COOL_DOWN)
##############################################################################################

In [5]:
##############################################################################################
# MODEL SELECTION HELP OR AVAILABLE MODELS
class Available_Models(Enum):
    VGG16 = "VGG16"
    VGG19 = "VGG19"
    InceptionV3 = "InceptionV3"
    Xception = "Xception" # TensorFlow ONLY
    ResNet50 = "ResNet50"
    ResNet18 = "ResNet18"
    ResNet34 = "ResNet34"
    DenseNet121 = "DenseNet121"
    DenseNet169 = "DenseNet169"
    DenseNet201 = "DenseNet201"
    MobileNet = "MobileNet"
    InceptionResNetV2 = "InceptionResNetV2"
    NASNetLarge = "NASNetLarge"
    NASNetMobile = "NASNetMobile"
    Lenet_Style = "Lenet_Style"
##############################################################################################

# Global Configurations
# Hyper-parameters Configuration etc

# Initialize all the the global parameters
C_Guage = None
ML_Model = None
P_ML_Model = None
TRAIN_GENERATOR = None
VALID_GENERATOR = None
STEP_SIZE_TRAIN = None
STEP_SIZE_VALID = None
Model_Name = None
Weights_Path = None
History_Path = None
Model_Path = None
History = None

# Initialize paths, these are all set in the next cell
# BASE = "."                               # Dont think this is needed
ROOT_DS_FOLDER_PATH = None                 # Dell_Path, root folder containing TRAINING, VALIDATION, TEST DIRECTORY
TRAINING_DIRECTORY_PATH = None             # Dell_path/TRAINING_DIRECTORY
VALIDATION_DIRECTORY_PATH = None           # Dell_path/VALIDATION_DIRECTORY
TESTING_DIRECTORY_PATH = None              # Dell_path/TEST_DIRECTORY
Store_Trained_ML_Model_Info_At = None      # ./Trained_Models/C_Guage

##### FUNCTION 1 INPUTS #####
OUTPUT_CLASSES = 9                                # Tell the model how many classes there are, This is set in MAIN LOOP
FINAL_LAYER_CLASSIFICATION_FUCNTION = "softmax"   # Sets the activation function for the output layer
POOLING = None                                    # avg pooling, max pooling, or None for flatten
INITIALIZATION_WIEGHTS = "imagenet"               # intialize the weights as the imagenet weights
INCLUDE_TOP = False
INPUT_TENSOR = None

##### FUNCTION 2 INPUTS #####

##### FUNCTION 3 INPUTS #####
FREEZE_EARLY_LAYERS_WIHT_PERCENT = 0

##### FUNCTION 4 INPUTS, Compile_Model() #####
# Make sure to use the GPUs if available when compiling the model
AVAILABLE_GPUs = len([gpu for gpu in device_lib.list_local_devices() if gpu.device_type=="GPU"])
if(AVAILABLE_GPUs == 0):
    AVAILABLE_GPUs = 1
    print("AVAILABLE_GPUs = 0 (NO GPU ACCESS)")
print("AVAILABLE_GPUs=",AVAILABLE_GPUs)

##### FUNCTION 5 INPUTS #####
SEED = 101
COLOR_MODE = "rgb"
SHUFFLE = True
CLASS_MAPPINGS = []              # set in MAIN LOOP
BATCH_SIZE = 0                   # Must be zero here 

##### FUNCTION 6 INPUTS #####

##### FUNCTION 7 INPUTS #####
Monitor = "val_acc"                # Monitor for maximium val_acc or Minimium val_loss
Mode = "auto"                      # max for val_acc and min for val_loss
EARLY_STOPPING_PATIENCE = 15       # if the model doesnt improve after this many epochs then the training stops
EPOCHs = 500                       # This is set high because we do early stopping

##### FUNCTION 8 INPUTS #####

##### FUNCTION 9 INPUTS #####

##### IMT INPUTS #####
COOL_DOWN = 25        # how long to sleep after training


AVAILABLE_GPUs= 1


In [1]:
###### SET THE INPUTS HERE  #########

# Path to the directory that contains TRAINING_DIRECTORY, VALIDATION_DIRECTORY, and TEST_DIRECTORY
Dell_Path = "/home/hikmat/Desktop/DLWSpace/FAA_Guages_Data/" # If training on the local machine
# CHANGE C_Guage in main loop for name

# MAIN FOR
LEARNING_RATE_LST = [0.001]     #, 0.0009]#, 0.005, 0.009]  -> learning_rate -> LEARNING_RATE (4)
Dropout_Rate_LST = [0.25]       #, 0.25]#, 0.5, 0.0]        -> dropout_rate -> Dropout_Rate (1)
Is_To_Apply_Data_Augmentation_Lt = [False]#, False]         -> l_Is_To_Apply_Data_Augmentation -> Is_To_Apply_Data_Agumentation (5)

# FUNCTION 1
Is_To_Add_Dropout = False             #True
Dense_Layers_Neurons = []             #[512] , tells how many neurons in the output dense layer
Dropout_Rate = None

# FUNCTION 2, Add_Conv_Regularizer()
REGULARIZE_EARLY_LAYERS_WIHT_PERCENT = 0
REGULARIZE_EARLY_LAYERS_WITH_RATIO = 0.001

# FUNCTION 4, Compile_Model()
LEARNING_RATE = None           # for setting the adam optimizer

# FUNCTION 5, Create_Data_Loader()
Is_To_Apply_Data_Augmentation = False


##################################
# TEST_CSV ... DO I NEED THIS?
TEST_CSV_PATH = None
TEST_CSV_Predicted_PATH = None
TEST_CSV = None
##################################


# Start the main loop
if __name__ == "__main__":

    for learning_rate, dropout_rate, l_Is_To_Apply_Data_Augmentation in zip(LEARNING_RATE_LST, 
                                                Dropout_Rate_LST, Is_To_Apply_Data_Augmentation_Lt):
        LEARNING_RATE = learning_rate
        Dropout_Rate = dropout_rate
        Is_To_Apply_Data_Augmentation = l_Is_To_Apply_Data_Augmentation
        
        # USED FOR NAMING THINGS?
        C_Guage = "HPE_Models"

        # Set the path to the trained models, if one doesn't exist -> create one
        Trained_Models_Root_Path = "./Trained_Models"
        if(os.path.exists(Trained_Models_Root_Path)):
            pass
        else:
            os.makedirs(Trained_Models_Root_Path)
        
        # Choose where to store the Model info, FUNCTION 6
        Store_Trained_ML_Model_Info_At = "{0}/{1}".format(Trained_Models_Root_Path, C_Guage)
        
        if(os.path.exists(Store_Trained_ML_Model_Info_At)):
            pass
        else:
            os.makedirs(Store_Trained_ML_Model_Info_At)

        print("*" * 50)
        print("*" * 25)
        print("HPE={0}".format(C_Guage))


        # Set the root path, in my case = to Dell_path
        ROOT_DS_FOLDER_PATH = "{0}".format(Dell_Path)
        
        #########
        # Might not need these
        #TEST_CSV_PATH  = "{1}/Training/CSV/{0}_ML_CSV_For_CNN_Test.csv".format(C_Guage, Dell_Path)
        #TEST_CSV_Predicted_PATH = "{1}/Training/CSV/{0}_ML_Predicted_CSV_For_CNN_Test_{2}.csv".format(
        #    C_Guage, Dell_Path, str(datetime.now()))
        #########

        # Set the path to the training directory
        TRAINING_DIRECTORY_PATH = "{0}/TRAINING_DIRECTORY".format(ROOT_DS_FOLDER_PATH)
        # Define the number of classes based on how many subdirectories are in the Training Folder
        OUTPUT_CLASSES = len(os.listdir(TRAINING_DIRECTORY_PATH))
        # Map the classes 0-8
        CLASS_MAPPINGS = []
        for label in range(0, OUTPUT_CLASSES):
            CLASS_MAPPINGS.append(str(label))
            
        # Set the path to the validation and test directories    
        VALIDATION_DIRECTORY_PATH = "{0}/VALIDATION_DIRECTORY".format(ROOT_DS_FOLDER_PATH)
        TESTING_DIRECTORY_PATH = "{0}/TESTING_DIRECTORY".format(ROOT_DS_FOLDER_PATH)
        
        # print the paths
        print("Current TRAINING_DIRECTORY_PATH={0}".format(TRAINING_DIRECTORY_PATH))
        print("Current VALIDATION_DIRECTORY_PATH={0}".format(VALIDATION_DIRECTORY_PATH))
        print("Current TESTING_DIRECTORY_PATH={0}".format(TESTING_DIRECTORY_PATH))
        
        ########
        # This could be for evaluation
        #print("PREDICTED_CSV_WILL_BE_SAVED_AT_PATH=" + str(TEST_CSV_Predicted_PATH))
        #print("TEST_CSV_PATH={0}".format(TEST_CSV_PATH))
        #TEST_CSV = pd.read_csv(TEST_CSV_PATH)
        #print("*" * 25)
        ########


        
###########################################################################################
# THIS SECTION OF THE CODE IS WHAT ACTUALLY RUNS SOMETHING
# MUST DEFINE BATCH_SIZE, INPUT_SIZE, SELECTED MODEL (from list of Available_Models)
# MAIN CODE IS IN THE Initiate_Model_Training FUNCTION
        
        BATCH_SIZE = 96 * AVAILABLE_GPUs
        INPUT_SIZE = (224,224, 3)
        SELECTED_MODEL = Available_Models.ResNet50
        Initiate_Model_Training()
        print("Saving Prediction of {0}".format(SELECTED_MODEL))

#             BATCH_SIZE = 1024 * AVAILABLE_GPUs
#             INPUT_SIZE = (64,64, 3)
#             SELECTED_MODEL = Available_Models.Lenet_Style
#             Initiate_Model_Training()

#             BATCH_SIZE = 96 * AVAILABLE_GPUs
#             INPUT_SIZE = (224,224, 3)
#             SELECTED_MODEL = Available_Models.VGG19
#             Initiate_Model_Training()
#             print("Saving Prediction of {0}".format(SELECTED_MODEL))
#             TEST_CSV.to_csv(TEST_CSV_Predicted_PATH)


#        BATCH_SIZE = 2048 * AVAILABLE_GPUs
#        # INPUT SIZE GOES TO FUNCTION 5
#        INPUT_SIZE = (64,64, 3)
#        SELECTED_MODEL = Available_Models.ResNet18
#        Initiate_Model_Training()
#        print("Saving Prediction of {0}".format(SELECTED_MODEL))
#        TEST_CSV.to_csv(TEST_CSV_Predicted_PATH)


        #BATCH_SIZE = 512 * AVAILABLE_GPUs
        #INPUT_SIZE = (128,128, 3)
        #SELECTED_MODEL = Available_Models.ResNet34
        #Initiate_Model_Training()
        #print("Saving Prediction of {0}".format(SELECTED_MODEL))
        #TEST_CSV.to_csv(TEST_CSV_Predicted_PATH)



#             BATCH_SIZE = 96 * AVAILABLE_GPUs
#             INPUT_SIZE = (224,224, 3)
#             SELECTED_MODEL = Available_Models.ResNet50
#             Initiate_Model_Training()
#             print("Saving Prediction of {0}".format(SELECTED_MODEL))
#             TEST_CSV.to_csv(TEST_CSV_Predicted_PATH)


#             BATCH_SIZE = 32 * AVAILABLE_GPUs
#             INPUT_SIZE = (299,299, 3)
#             SELECTED_MODEL = Available_Models.Xception
#             Initiate_Model_Training()
#             print("Saving Prediction of {0}".format(SELECTED_MODEL))


#             BATCH_SIZE = 32 * AVAILABLE_GPUs
#             INPUT_SIZE = (299,299, 3)
#             SELECTED_MODEL = Available_Models.InceptionV3
#             Initiate_Model_Training()

#             BATCH_SIZE = 32 * AVAILABLE_GPUs
#             INPUT_SIZE = (299,299, 3)
#             SELECTED_MODEL = Available_Models.InceptionResNetV2
#             Initiate_Model_Training()

#             BATCH_SIZE = 32 * AVAILABLE_GPUs
#             INPUT_SIZE = (224,224, 3)
#             SELECTED_MODEL = Available_Models.DenseNet121
#             Initiate_Model_Training()

#             BATCH_SIZE = 96 * AVAILABLE_GPUs
#             INPUT_SIZE = (224,224, 3)
#             SELECTED_MODEL = Available_Models.VGG16
#             Initiate_Model_Training()

#             print("SAVING_PREDICTED_CSV_TO_PATH=" + str(TEST_CSV_Predicted_PATH))
        #TEST_CSV.to_csv(TEST_CSV_Predicted_PATH)
        #TEST_CSV = None
        #TEST_CSV_PATH = None
        #TEST_CSV_Predicted_PATH = None
        #print("*" * 25)
        print("{0} Training Has Completed!".format(C_Guage))
#         break

NameError: name 'os' is not defined